In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
# loading the dataset to a Pandas DataFrame
credit_card_data = pd.read_csv('../input/creditcardfraud/creditcard.csv')
credit_card_data.head()
# original column names are hidden
# Class = {'0':'Legit Transaction', '1':'Fraudulent Transaction'}

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
credit_card_data.shape

(284807, 31)

In [6]:
# checking for missing values
credit_card_data.isnull().sum().sum() # there is no missing value

0

In [7]:
# distribution of legit transactions & fraudulent transactions
credit_card_data['Class'].value_counts()

# data is highly imbalance

0    284315
1       492
Name: Class, dtype: int64

In [8]:
print((492/(284315+492)) * 100)
# only 0.17% of rows represent Fraudulent transactions

0.1727485630620034


In [9]:
# comparing the values for both transactions
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


### We would go with under-sampling

In [11]:
# separating the data for analysis
legit = credit_card_data.loc[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1] # n=492

In [12]:
legit_sample = legit.sample(n=492) # randomly selecting 492 samples

In [13]:
credit_card_data_underS = pd.concat([legit_sample, fraud], axis=0)

In [17]:
print(credit_card_data_underS.shape)

credit_card_data_underS['Class'].value_counts()

(984, 31)


0    492
1    492
Name: Class, dtype: int64

In [24]:
credit_card_data_underS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 984 entries, 214073 to 281674
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    984 non-null    float64
 1   V1      984 non-null    float64
 2   V2      984 non-null    float64
 3   V3      984 non-null    float64
 4   V4      984 non-null    float64
 5   V5      984 non-null    float64
 6   V6      984 non-null    float64
 7   V7      984 non-null    float64
 8   V8      984 non-null    float64
 9   V9      984 non-null    float64
 10  V10     984 non-null    float64
 11  V11     984 non-null    float64
 12  V12     984 non-null    float64
 13  V13     984 non-null    float64
 14  V14     984 non-null    float64
 15  V15     984 non-null    float64
 16  V16     984 non-null    float64
 17  V17     984 non-null    float64
 18  V18     984 non-null    float64
 19  V19     984 non-null    float64
 20  V20     984 non-null    float64
 21  V21     984 non-null    float64

In [19]:
# splitting data into dependant and independant variables
X = credit_card_data_underS.drop(columns='Class', axis=1)
Y = credit_card_data_underS['Class']

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)
# stratify helps to distribute the data in equal proportion from each group

### Model building using Logistic Regression

In [27]:
model = LogisticRegression(solver='liblinear')
# AttributeError: 'str' object has no attribute 'decode' with solver 'lbfgs'

In [28]:
# training the Logistic Regression Model with Training Data
model.fit(X_train, Y_train)


LogisticRegression(solver='liblinear')

### Model Evaluation

In [29]:
# accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [30]:
print(f'Accuracy score on Test Data : {test_data_accuracy}')

Accuracy score on Test Data : 0.9137055837563451
